In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from kine2D import kine2D
from net import Net2D as Net
# 定义训练参数
param = {"Net":"Net2D",
         "generate_num":20000,
         "tran_num":20000,
         "test_num":100,
         "lr":0.01}

In [2]:
# 定义模型、损失函数和优化器
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), param["lr"])

In [7]:
# 生成随机数据
def generate_data(num_samples):
    inputs = []
    labels = []

    for _ in range(num_samples):
        theta = np.array([
            np.random.normal(loc=0, scale=np.pi/4),
            np.random.normal(loc=0, scale=np.pi/3),
            np.random.normal(loc=0, scale=np.pi/2)
        ])

        curves = kine2D(theta)
        labels.append(theta)
        inputs.append([curves[-1][0][-1],curves[-1][1][-1]])
    inputs = np.array(inputs)  # 先转换为 NumPy 数组
    labels = np.array(labels)  # 先转换为 NumPy 数组
    return torch.tensor(inputs, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)
# 准备数据
inputs, labels = generate_data(param["generate_num"])


In [8]:
print(inputs[0:3])
print(labels[0:3])

tensor([[-2.2528,  0.5652],
        [-0.9881,  2.5808],
        [ 0.2757,  2.4746]])
tensor([[-0.8942,  1.0617,  0.1831],
        [-0.4283,  0.5883, -1.7330],
        [ 0.9408,  0.9902,  1.2050]])


In [ ]:
# 训练
def train_model(model, criterion, optimizer, inputs, labels, num_epochs):
    data = []
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
            data.append(loss.item())
    # 创建一个折线图
    plt.figure(figsize=(8, 4))  # 图片大小
    plt.plot(data, marker='o', linestyle='-', color='b')  # 点样式、线样式、颜色
    plt.xlabel('epoch')  # x轴标签
    plt.ylabel('LOSS')  # y轴标签
    plt.grid(True)  # 显示网格
    plt.show()

train_model(model, criterion, optimizer, inputs, labels, param["tran_num"])

# 保存模型
torch.save(model.state_dict(), f'model_libs/{param["Net"]}_model.pth')


In [ ]:
# 测试模型
test_inputs, test_labels = generate_data(param["test_num"])
with torch.no_grad():
    test_outputs = model(test_inputs)
    test_loss = criterion(test_outputs, test_labels)
print(f'Test Loss: {test_loss.item()}')

2D化，首先是直接theta拟合，问题是这里的loss到底是什么呢？

好像并没有改进什么